#TODO: Maybe try this - https://youtu.be/0VLAoVGf_74?si=zuJ8AL_wLbsbRdd5
Compare MHA, MQA, MLA (DeepSeek)

## Part 2: Apply TransformerClassifier (Encoder Only) 

In [ ]:
from common_utils import *
from transformer import TransformerClassifier
from datasets import load_dataset

In [ ]:
wassa_dataset = load_dataset("csv", data_files="./dataset/text_emotion.csv")
wassa_dataset = wassa_dataset.rename_column('content', 'text')
wassa_dataset_dict = create_train_validation_test(wassa_dataset['train'])

embedding_matrix = np.load(EMBEDDING_PATH)

with open(WORD2IDX_PATH, "r", encoding="utf-8") as f:
    word2idx = json.load(f)